# Excercise 6
## Question 1
(a) Show empirically that the information limit of 2 prediction bits per parameter also holds for nearest neighbors


In [76]:
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier # import knn from sklearn


In [43]:
X = np.random.rand(1000, 1)
y = np.random.choice([0, 1], 1000)

In [58]:
k = 1
model = KNeighborsClassifier(n_neighbors=k)
model.fit(X, y)


KNeighborsClassifier(n_neighbors=1)

In [59]:
predictions = model.predict(X)
accuracy = np.mean(predictions == y)
print(accuracy)

1.0


In [62]:
num0 = np.sum(predictions == 0)
num1 = np.sum(predictions == 1)
p = [num0/(num0+num1), num1/(num0+num1)]
print(num0, num1)

503 497


In [63]:
H = -np.sum([pi * np.log2(pi) for pi in p]) * len(y)
print(H)
print(H/len(y), "< 2, so the information limit holds")


999.9740313334506
0.9999740313334506 < 2, so the information limit holds


(b) Extend your experiments to multi-class classification.

In [70]:
X = np.random.rand(1000, 1)
y = np.random.choice([0, 1, 2], 1000)

In [71]:
k = 1
model = KNeighborsClassifier(n_neighbors=k)
model.fit(X, y)


KNeighborsClassifier(n_neighbors=1)

In [72]:
predictions = model.predict(X)
accuracy = np.mean(predictions == y)
print(accuracy)

1.0


In [73]:
nums = [np.sum(predictions == i) for i in range(3)]
p = [num/len(y) for num in nums]
print(nums)

[344, 337, 319]


In [77]:
H = -np.sum([pi * math.log(pi, len(p)) for pi in p]) * len(y)
print(H)
print(H/len(y), "< 1.5, so the information limit holds")


999.5433792751933
0.9995433792751933 < 1.5, so the information limit holds


## Question 2 Finite State Machine Generalization
(a) Implement a program that automatically creates a set of if then clauses from the training table of a binary dataset of your choice. Implement different strategies to minimize the number of if-then clauses. Document your strategies, the number of resulting conditional clauses, and the accuracy achieved.


In [79]:
import pandas as pd


In [88]:
# Banana quality dataset
dataset = pd.read_csv("datasets/banana_quality.csv")
dataset["result"] = dataset["Quality"].apply(lambda x: 1 if x == "Good" else 0)
X = dataset[["Size", "Weight", "Sweetness", "HarvestTime", "Ripeness", "Acidity"]].values
y = dataset["result"].values
X.shape, y.shape

((8000, 6), (8000,))

In [91]:
# Baseline Approach: Decision Tree
from sklearn.tree import DecisionTreeClassifier

def train_decision_tree(X, y):
    model = DecisionTreeClassifier()
    model.fit(X, y)
    return model

model = train_decision_tree(X, y)
model.score(X, y), model.tree_.node_count

(1.0, 1115)

In [98]:
# Unique values for each feature 
def unique_feature_value_strategy(X, y):
    def apply_rules(rules, X):
        y_pred = np.zeros(X.shape[0])
        for i, value, pred in rules:
            y_pred[X[:, i] == value] = pred
        return y_pred
    rules = []
    for i in range(X.shape[1]):  # Iterate over each feature
        unique_values = np.unique(X[:, i])
        for value in unique_values:
            if np.mean(y[X[:, i] == value]) > 0.5:  # Majority class
                rules.append((i, value, 1))
            else:
                rules.append((i, value, 0))
    return rules, apply_rules



rules, apply_rules = unique_feature_value_strategy(X, y)
y_pred = apply_rules(rules, X)

np.mean(y_pred == y), len(rules)


(1.0, 48000)

In [140]:
# decision tree from scratch

class Node:
    def __init__(self, feature=None, value=None, left=None, right=None, threshold=None):
        self.feature = feature
        self.value = value
        self.left = left
        self.right = right
        self.threshold = threshold

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.root = None
    
    def fit(self, X, y):
        self.root = self._grow_tree(X, y, depth=0)
    
    
    def _entropy(self, y):
        p = np.array([np.mean(y == c) for c in np.unique(y)])
        return -np.sum(p * np.log2(p))
    
    def _information_gain(self, X, y, feature, threshold):
        left = y[X[:, feature] < threshold]
        right = y[X[:, feature] >= threshold]
        n = len(y)
        y_left = len(left) / n * self._entropy(left)
        y_right = len(right) / n * self._entropy(right)
        return self._entropy(y) - (y_left + y_right)
        
    
    def _grow_tree(self, X, y, depth):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))
        if (self.max_depth is not None and depth >= self.max_depth) or n_classes == 1 or n_samples < 2:
            value = int(np.mean(y) > 0.5)
            return Node(value=value)
        best_gain = 0
        best_feature = None
        best_threshold = None
        # calculate the information gain for each feature
        for feature in range(n_features):
            unique_values = np.unique(X[:, feature])
            for value in unique_values:
                gain = self._information_gain(X, y, feature, value)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = value
        
        if best_gain <= 0:
            value = int(np.mean(y) > 0.5)
            return Node(value=value)
        else:
            left_indices = X[:, best_feature] < best_threshold
            right_indices = X[:, best_feature] >= best_threshold
            left = self._grow_tree(X[left_indices], y[left_indices], depth + 1)
            right = self._grow_tree(X[right_indices], y[right_indices], depth + 1)
            return Node(feature=best_feature, threshold=best_threshold, left=left, right=right)
        
    def n_nodes(self):
        def _n_nodes(node):
            if node is None:
                return 0
            return 1 + _n_nodes(node.left) + _n_nodes(node.right)
        return _n_nodes(self.root)
        
    def _predict_single(self, node, X):
        if node.value is not None:
            return node.value

        if X[node.feature] < node.threshold:
            return self._predict_single(node.left, X)
        else:
            return self._predict_single(node.right, X)
    def predict(self, X):
        return [self._predict_single(self.root, X[i]) for i in range(X.shape[0])]
    


model = DecisionTree(max_depth=None)
model.fit(X, y)


In [142]:
y_pred = model.predict(X)
np.mean(y_pred == y), model.n_nodes()

(1.0, 1013)

I used three methods. 
The first one is sklearn's decision tree; 

The second one:
Iterate through every feature, memorize all the unique values, and use them as clauses such that if more y == 1 have that unique values, predict it as 1 else 0.

The third one:
A decision tree from scratch. At each node, find the unique value of the feature that yields the highest information gain, and use them as the split for the node.

1. number of clauses: 1115; acc: 1.0
2. number of clauses: 48000; acc: 1.0
3. number of clauses: 1013; acc: 1.0




(b) Use the algorithms developed in (a) on different datasets. Again, observe how your choices make a difference.

In [164]:
# Heart Disease Dataset
dataset = pd.read_csv("datasets/heart.csv")
X = dataset.drop("output", axis=1).values
y = dataset["output"].values

In [165]:
def check_methods(X, y):
    model = DecisionTreeClassifier()
    model.fit(X, y)
    print("SKLearn Decision Tree:", model.score(X, y), model.tree_.node_count)
    model = DecisionTree(max_depth=None)
    model.fit(X, y)
    print("Decision Tree from scratch:", np.mean(model.predict(X) == y), model.n_nodes())
    rules, apply_rules = unique_feature_value_strategy(X, y)
    y_pred = apply_rules(rules, X)
    print("Unique Feature Value Strategy:", np.mean(y_pred == y), len(rules))
    print("Number of samples:", X.shape[0])
    

In [166]:
check_methods(X, y)

SKLearn Decision Tree: 1.0 91
Decision Tree from scratch: 1.0 91
Unique Feature Value Strategy: 0.7656765676567657 398
Number of samples: 303


In [167]:
dataset = pd.read_csv("datasets/water_potability.csv").dropna()
X = dataset.drop("Potability", axis=1).values
y = dataset["Potability"].values

In [163]:
check_methods(X, y)

SKLearn Decision Tree: 1.0 775


Decision Tree from scratch: 1.0 755
Unique Feature Value Strategy: 1.0 18099
Number of samples: 2011


Hand-written Decision Tree has used 94 and 755 clauses in respective dataset - it uses the smallest amount of clauses
The unique feature value strategy uses the most number of clauses, but achieves the lowest accuracy


(c) Finally, use the programs developed in (a) on a completely random dataset, generated artificially. Vary your strategies but also the number of input columns as well as the number of instances. How many if-then clauses do you need?

In [172]:
def generate_artificial_data(n_samples, n_features):
    X = np.random.rand(n_samples, n_features)
    y = np.random.randint(0, 2, n_samples)
    return X, y


In [173]:
X, y = generate_artificial_data(1000, 10)
check_methods(X, y)

SKLearn Decision Tree: 1.0 439
Decision Tree from scratch: 1.0 425
Unique Feature Value Strategy: 1.0 10000
Number of samples: 1000


In [174]:
X, y = generate_artificial_data(1000, 20)
check_methods(X, y)

SKLearn Decision Tree: 1.0 385
Decision Tree from scratch: 1.0 361
Unique Feature Value Strategy: 1.0 20000
Number of samples: 1000


In [175]:
X, y = generate_artificial_data(500, 10)
check_methods(X, y)

SKLearn Decision Tree: 1.0 219
Decision Tree from scratch: 1.0 209
Unique Feature Value Strategy: 1.0 5000
Number of samples: 500


## Question 3
(a) Create a long random string using a Python program, and use a lossless compression algorithm of your choice to compress the string. Note the compression ratio

In [1]:
import random
import string
import zlib


In [5]:
random_string = ''.join(random.choices(string.ascii_letters  + string.digits, k=100000))
random_string

'...'

In [6]:

compressed_string = zlib.compress(random_string.encode())
size = len(compressed_string)

In [7]:
print("Original length:", len(random_string))
print("Compressed length:", len(compressed_string))
print("Compression ratio:",  len(random_string)/len(compressed_string))


Original length: 100000
Compressed length: 75202
Compression ratio: 1.329751868301375


(b) What is the expected compression ratio in (a)? Explain why?

The compression ratio achieved by lossless compression algorithms like zlib depends on the redundancy and patterns present in the data. The more redundancy and patterns, the higher the compression ratio. The expected compression ratio should be 1:1, as the string is random and has no redundancy or patterns.
However, even though we are generating a random string, there might still be some sequences or patterns that could be compressed. For instance, if the random string contains repeating substrings, the compression algorithm might be able to exploit this redundancy and achieve a compression ratio greater than 1:1.